### 데이터 불러오기

In [ ]:
# 모델 불러오기
from gensim.models import word2vec
model_name = '150features_30minwords_5context'
w2v_model = word2vec.Word2Vec.load(model_name)

In [ ]:
import pandas as pd
import numpy as np

# 단어 - 감정 사전 파일 열기
emotions = pd.read_csv('emotion.csv')

# 3점척도 -> 0~1사이의 점수로 변경
emotions.iloc[:, 1:] = emotions.iloc[:, 1:]/3
print(emotions.head())

In [ ]:
# wv_array 작성
wv_list = []
for w in emotions['word']:
    wv_list.append(w2v_model.wv[w])
wv_arr = np.array(wv_list)
wv_arr[:5, :5]

### 케라스 모델 작성

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
import numpy as np

# 학습데이터 설정
X_train = wv_arr
Y_train = np.array(emotions.iloc[:, 1:])

# 모델 리스트 준비
models = list()
for i in range(5):
    
    # 모델 구성
    model = Sequential()
    model.add(Dense(input_dim=150, units=150, activation='relu'))
    model.add(Dropout(rate=0.3))
    model.add(Dense(units=150, activation='relu'))
    model.add(Dropout(rate=0.3))
    model.add(Dense(units=8, activation='softmax'))

    # 모델 옵션       
    model.compile(optimizer=Adam(lr=0.001), loss='cosine_proximity', metrics=['accuracy'])

    # 모델 학습
    model.fit(X_train, Y_train, epochs=250, batch_size=50)
    models.append(model)

In [ ]:
models[0].predict(np.array([w2v_model['ㅋㅋ']]))

In [ ]:
# 모델 저장하기
from keras.models import load_model
for i, m in enumerate(models):
    m.save('./model/ann_model{}.h5'.format(i))

In [ ]:
# 모델 불러오기
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
import numpy as np

from keras.models import load_model
import os
models = list()  # 불러온 모델을 담을 리스트
path_dir = './model/'  # 파일 위치 경로
file_list = os.listdir(path_dir)  # 파일 리스트
for fl in file_list:
    if fl.find('.h5') is not -1:  # 파일명에 '.h5'가 있으면
        models.append(load_model(path_dir + fl))

In [ ]:
# 첫번째 모델의 계산 결과
models[0].predict(np.array([w2v_model['졸리다']]))

In [ ]:
# 단어-감정 점수산출 함수
def ens_emotion(models, word = '좋다'):
    ens_result = np.zeros([8], dtype=np.float32)
    for m in models:
        ens_result = ens_result + m.predict(np.array([w2v_model[word]]))
    ens_result = ens_result / len(models)
    return ens_result

word_emo = ens_emotion(models, '나간다')
emo_list = ['joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust', 'anger', 'anticipation']
for i in range(len(emo_list)):
    print('{}: {}'.format(emo_list[i], word_emo[0, i]/word_emo.sum()))